# Notebook to calculate satellite overpasses for an AOI and date range

In [1]:
!pip install skyfield

  Created wheel for skyfield: filename=skyfield-1.26-py3-none-any.whl size=327095 sha256=f6b6fb59b7f4cb925b18f4bf07960f751d6fdfedca8f27cae7121e3a82bbc3ca
  Stored in directory: c:\users\bra467\appdata\local\pip\cache\wheels\1f\4c\62\f2b69f8727cf9097c319cf17375def63bc13e539a4659549c9
  Created wheel for jplephem: filename=jplephem-2.15-py3-none-any.whl size=46024 sha256=0465f6a5c712cff9d0ae222dfe722ebdc7ba75a965776438a22c215430b0d6ca
  Stored in directory: c:\users\bra467\appdata\local\pip\cache\wheels\61\c7\45\352389757a3b6c9ddfe03bcb599bca17b42dcc99d2a1db3a0c
Successfully built skyfield jplephem


In [1]:
from skyfield.api import Topos, load
import datetime

Loaded 2956 satellites


In [42]:
stations_url = 'http://celestrak.com/NORAD/elements/active.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')

Loaded 2956 satellites


### Query the list of satellites for platforms of interest

In [49]:
query = ['SENTINEL-2','SENTINEL-3','WORLDVIEW-2','LANDSAT-8','ICESAT-2','PLEIADES 1','SPOT','GEOEYE']
by_name = {sat.name: sat for sat in satellites}
results = [{k:v for k,v in by_name.items() if k.startswith(p)} for p in query]
platform_sats = {}
for d in results:
    platform_sats.update(d)
platform_sats

{'SENTINEL-2A': <EarthSatellite 'SENTINEL-2A' number=40697 epoch=2020-09-04T01:47:34Z>,
 'SENTINEL-2B': <EarthSatellite 'SENTINEL-2B' number=42063 epoch=2020-09-04T02:37:53Z>,
 'SENTINEL-3A': <EarthSatellite 'SENTINEL-3A' number=41335 epoch=2020-09-04T02:45:38Z>,
 'SENTINEL-3B': <EarthSatellite 'SENTINEL-3B' number=43437 epoch=2020-09-03T19:22:19Z>,
 'WORLDVIEW-2 (WV-2)': <EarthSatellite 'WORLDVIEW-2 (WV-2)' number=35946 epoch=2020-09-04T01:20:33Z>,
 'ICESAT-2': <EarthSatellite 'ICESAT-2' number=43613 epoch=2020-09-03T20:54:16Z>,
 'PLEIADES 1A': <EarthSatellite 'PLEIADES 1A' number=38012 epoch=2020-09-04T01:50:58Z>,
 'PLEIADES 1B': <EarthSatellite 'PLEIADES 1B' number=39019 epoch=2020-09-02T17:44:40Z>,
 'SPOT 6': <EarthSatellite 'SPOT 6' number=38755 epoch=2020-09-04T03:50:42Z>,
 'SPOT 7': <EarthSatellite 'SPOT 7' number=40053 epoch=2020-09-04T04:41:37Z>,
 'GEOEYE 1': <EarthSatellite 'GEOEYE 1' number=33331 epoch=2020-09-04T02:44:34Z>}

In [50]:
aquisition_angle = {'WORLDVIEW-2':26,'SPOT':26,'PLEIADES':26}

In [51]:
ts = load.timescale()
utc_offset = 8
jurien = Topos('30.316650 S', '115.015349 W')
t0 = ts.utc(2020, 10, 1)
t1 = ts.utc(2020, 10, 30)

In [52]:
overpasses = {}
for name,satellite in platform_sats.items():
    ak = [k for k in aquisition_angle.keys() if name.startswith(k)]
    if len(ak) == 1:
        ang = aquisition_angle[ak[0]]
    else:
        ang=5

    altitude = 90-ang
    t, events = satellite.find_events(jurien, t0, t1, altitude_degrees=altitude)
    for ti, event in zip(t, events):
        ev = (f'{name} rise above {altitude}°', f'{name} culminate', f'{name} set below {altitude}°')[event]
        dt = ti.utc_datetime() + datetime.timedelta(hours=8)
        if (dt.hour > 8) & (dt.hour < 16):
            overpasses[dt.strftime('%Y-%m-%d %H:%M:%S') + f' +{utc_offset}'] = ev
overpasses = dict(sorted(overpasses.items(), key=lambda item: item[0]))
for t,e in overpasses.items():
    print(f'{t} {e}')

2020-10-01 13:58:59 +8 SENTINEL-3A rise above 85°
2020-10-01 13:59:07 +8 SENTINEL-3A culminate
2020-10-01 13:59:21 +8 SENTINEL-3A set below 85°
2020-10-01 14:01:47 +8 SPOT 6 rise above 64°
2020-10-01 14:02:30 +8 SPOT 6 culminate
2020-10-01 14:03:12 +8 SPOT 6 set below 64°
2020-10-01 14:17:32 +8 WORLDVIEW-2 (WV-2) rise above 64°
2020-10-01 14:18:15 +8 WORLDVIEW-2 (WV-2) culminate
2020-10-01 14:18:59 +8 WORLDVIEW-2 (WV-2) set below 64°
2020-10-01 14:30:42 +8 PLEIADES 1B rise above 64°
2020-10-01 14:30:49 +8 SENTINEL-2A rise above 85°
2020-10-01 14:30:58 +8 SENTINEL-2A culminate
2020-10-01 14:31:08 +8 SENTINEL-2A set below 85°
2020-10-01 14:31:27 +8 PLEIADES 1B culminate
2020-10-01 14:32:13 +8 PLEIADES 1B set below 64°
2020-10-02 13:55:59 +8 SPOT 7 rise above 64°
2020-10-02 13:56:43 +8 SPOT 7 culminate
2020-10-02 13:57:27 +8 SPOT 7 set below 64°
2020-10-02 14:23:21 +8 PLEIADES 1A rise above 64°
2020-10-02 14:23:59 +8 PLEIADES 1A culminate
2020-10-02 14:24:38 +8 PLEIADES 1A set below 64°
2